In [1]:
df = spark.sql("SELECT * FROM bing_link_db.tbl_latest_news")
display(df)

StatementMeta(, eb216709-6145-4e5f-aecf-1abd8c39ee08, 3, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7baa2472-9ed4-4695-95c9-35d0e6f2536b)

In [2]:
import synapse.ml.core
from synapse.ml.services import AnalyzeText

StatementMeta(, eb216709-6145-4e5f-aecf-1abd8c39ee08, 4, Finished, Available, Finished)

In [3]:
# Import the model and configure input and output columns

model = (AnalyzeText()
        .setTextCol('description')
        .setKind('SentimentAnalysis')
        .setOutputCol('response')
        .setErrorCol('error'))

StatementMeta(, eb216709-6145-4e5f-aecf-1abd8c39ee08, 5, Finished, Available, Finished)

In [4]:
#apply the model to our df
result = model.transform(df)

StatementMeta(, eb216709-6145-4e5f-aecf-1abd8c39ee08, 6, Finished, Available, Finished)

In [6]:
display(result)

StatementMeta(, eb216709-6145-4e5f-aecf-1abd8c39ee08, 8, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, fb88c726-8c1a-4b78-961e-56820e4c73ee)

In [7]:
#create sentiment column
from pyspark.sql.functions import col

sentient_df = result.withColumn('sentiment', col('response.documents.sentiment'))

display(sentient_df)

StatementMeta(, eb216709-6145-4e5f-aecf-1abd8c39ee08, 9, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7bdb6242-4292-4ff2-98ea-eb9c080dca64)

In [8]:
# Using sql type 1 condition to update my existing table
from pyspark.sql.utils import AnalysisException

try:
    table_name = 'bing_link_db.tbl_sentiment_analysis'
    sentient_df.write.format('delta').saveAsTable(table_name)

except AnalysisException:
    print('Table Already Exist')
    sentient_df.createOrReplaceTempView('vw_sentient_df')

    spark.sql(f""" Merge into {table_name} target_table
                   USING vw_sentient_df source_view
                   ON source_view.url = target_table.url

                   WHEN MATCHED AND
                   source_view.title <> target_table.title OR
                   source_view.description <> target_table.description OR
                   source_view.category <> target_table.category OR
                   source_view.image <> target_table.image OR
                   source_view.provider <> target_table.provider OR
                   source_view.publishedDate <> target_table.publishedDate OR

                   THEN UPDATE SET *

                   WHEN NOT MATCHED THEN INSERT *
                
                """)

StatementMeta(, eb216709-6145-4e5f-aecf-1abd8c39ee08, 10, Finished, Available, Finished)